In [ ]:
from pymavlink import mavutil
import time

link = mavutil.mavlink_connection('COM11', baud=19200)

time.sleep(1)  #Give some time for serial to wake up

# https://mavlink.io/en/messages/common.html#COMMAND_LONG
# https://mavlink.io/en/messages/common.html#MAV_CMD_CONTROL_HIGH_LATENCY
# Start the High Latency 2 message
link.mav.command_long_send(1,0,mavutil.mavlink.MAV_CMD_CONTROL_HIGH_LATENCY,0,1,0,0,0,0,0,0)

In [ ]:
# Receive the message, parse and log it to a file
while 1:
    msg = link.recv_match(type='HIGH_LATENCY2',blocking=True)
    if not msg:
        break
    if msg.get_type() == "BAD_DATA":
        if mavutil.all_printable(msg.data):
            sys.stdout.write(msg.data)
            sys.stdout.flush()
    else:
        print(f'Altitude: {msg.altitude}, Battery: {msg.battery}')
        msgJSON = msg.to_json()

        f = open("hl_log.txt", "a")
        f.write(msgJSON)
        f.write('\n')
        f.close()

In [ ]:
# Analysis of the file
import json

data = []
with open('hl_log.txt') as f:
    for line in f:
        data.append(json.loads(line))

import pandas as pd

df = pd.DataFrame.from_records(data)